In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
#Calculate the date 1 year ago from the last data point in the database
    #Find the last date in the database
session.query(Measurement.date).order_by(Measurement.date.desc()).first()
    
    # Query for the date we'd need to get the last 12 months worth of data
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)

    #2016-08-23 is our starting point

In [ ]:

# Inspect the Measurement Table. Create the inspector and connect it to the engine
inspector = inspect(engine)

#Look at the column names in the Measurement table
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Use `engine.execute` to select and display the all rows from the Measurement table
# 0=ID 1=Station 2=Date 3=Prcp 4=tobs
allrows = engine.execute('SELECT date, prcp FROM Measurement WHERE date > 2016-08-23').fetchall()

#Turn the Measurement data into a Dataframe, set index to Date, sort by Date
prcp_df = pd.DataFrame(allrows, columns=['Date', 'Precipitation'])
prcp_df.set_index('Date', inplace=True, )
prcp_df.sort_values('Date')
prcp_df.head()


In [ ]:
#Plot the precipication data
prcp_df.plot()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()

In [ ]:
#Query to show how many stations are available in this dataset (there are 9)
station_count = engine.execute('SELECT COUNT(station) FROM station').fetchall()
station_count

In [ ]:
#What are the most active stations? 
station_activity = engine.execute('SELECT COUNT(station), station FROM Measurement WHERE date > 2016-08-23 GROUP BY station').fetchall()
station_activity_df = pd.DataFrame(station_activity, columns=['Observation Count', 'Station'])
station_activity_df.set_index('Station', inplace=True, )


In [ ]:
#Display station activity in descending order
station_activity_df.sort_values('Observation Count', ascending=False)

##Station USC00519281 has the highest observation count @2772

In [ ]:
##calculate lowest temp recorded, highest temp recorded, average temp recorded for station USC00519281
#Select tobs for station USC00519281 and put them into a dataframe

temprows = engine.execute('SELECT tobs, station FROM Measurement WHERE date > 2016-08-23').fetchall()
temper_df = pd.DataFrame(temprows, columns=['tobs', 'Station'])
temp_df = temper_df.loc[temper_df['Station'] == 'USC00519281']
temp_df.head()

In [ ]:
#Retrieve the min, max, and average tobs for this station
#min
temp_df.min()

In [ ]:
#max
temp_df.max()

In [ ]:
#average
temp_df.mean()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

temprows = engine.execute('SELECT tobs, station FROM Measurement WHERE date > 2016-08-23').fetchall()
temp_df_count = pd.DataFrame(temprows, columns=['tobs', 'Station'])
temp_df_count.groupby('Station').count()

In [ ]:
#Plot using the station with the highest number of observations (USC00513117)
most_temp_df = temper_df.loc[temper_df['Station'] == 'USC00513117']
most_temp_df.head()

In [ ]:
#plot using a histogram

most_temp_df.hist(bins=12)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

trip_temps = calc_temps('2016-07-05', '2016-07-11')
trip_temps

In [ ]:

tempdata = [62.0, 69.57, 74.0]
x_axis = np.arange(len(tempdata))
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, ["Min Temp", "Avg Temp", "Max Temp"])

plt.title("Vacation Week Temps (year prior) 7/5/16 - 7/11/16")
plt.ylabel("Tobs")


plt.bar(x_axis, tempdata, color='lightgreen', alpha=0.5, align="center")


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

tempdata = [69.57]
x_axis = np.arange(len(tempdata))
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, ["Avg Temp"])

plt.title('Average Temp')
plt.ylabel("Tobs")

error = [tempdata]

plt.bar(x_axis, tempdata, yerr=error, color='lightgreen', width=15, alpha=0.5, align="center")

In [ ]:
##Setting up flask to create routes

from flask import Flask, jsonify


In [ ]:
# Flask Setup
def create_app():
    app = Flask(__name__)

    return app

In [ ]:
## Design Flask APIs based on the queries from above
##precipitation for 12 months from Measurement table

##Query1: finding precipitation for 12 months from the end data point in the data-set
@app.route("/api/v1.0/precipitation")
def precip_func():
    """Return a list of precipitation data for dates in data > 8/23/2016"""
    # Query Measurement Table
    precip_results = session.query(measurement.precipitation > 2016-8-23).all()
    
    precip_list = []
    
    for precipitation, date in results:
        prcp_measurement = {}
        precipitation_dict["prcp"] = prcp
        date_dict["date"] = date
        results.append(prcp_measurement)

        return jsonify(prcp_measurement)
    
        if __name__ == '__precipitation__':
            app.run(debug=True)       

In [ ]:
##Query2: How many unique stations are in the dataset?

@app.route("/api/v1.0/stations")

def st_count_funct():
    """Return the number of stations in the data"""
    #Query station table
    st_results = session.query(Station.station).all()
    
    return jsonify(st_results)

    if __name__ == '__stations__':
        app.run(debug=True)       
        

In [ ]:
##Query3: Temperature data for the most recent 12 months in the data set

@app.route("/api/v1.0/tobs")

def st_count_funct():
    """Tobs for """
    #Query station table
tobs_results = session.query(measurement.tobs, measurement.date > 2016-8-23).all()
    
    return jsonify(tob_results)

if __name__ == '__main__':
    app.run(debug=True)